# Лабораторная работа №13 по PyTorch

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
# Загрузка датасета MNIST
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_loader = DataLoader(datasets.MNIST("data", train=True, download=True, transform=transform), batch_size=64, shuffle=True)

In [3]:
# Создание класса для перцептрона
class Perceptron(nn.Module):
    def __init__(self):
        super(Perceptron, self).__init__()
        self.fc = nn.Linear(28*28, 10)
    
    def forward(self, x):
        x = torch.sigmoid(self.fc(x.view(x.size(0), -1)))
        return x

In [4]:
# Инициализация модели, функции потерь и оптимизатора
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Perceptron().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [5]:
# Обучение модели
epochs = 10
for epoch in range(epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device) 
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(f'Epoch {epoch+1}, Batch {batch_idx}, Loss: {loss.item()}')

Epoch 1, Batch 0, Loss: 2.336301565170288
Epoch 1, Batch 100, Loss: 1.9606857299804688
Epoch 1, Batch 200, Loss: 1.8519337177276611
Epoch 1, Batch 300, Loss: 1.834634780883789
Epoch 1, Batch 400, Loss: 1.7609364986419678
Epoch 1, Batch 500, Loss: 1.7135716676712036
Epoch 1, Batch 600, Loss: 1.737916350364685
Epoch 1, Batch 700, Loss: 1.6795607805252075
Epoch 1, Batch 800, Loss: 1.6690044403076172
Epoch 1, Batch 900, Loss: 1.701741099357605
Epoch 2, Batch 0, Loss: 1.7163619995117188
Epoch 2, Batch 100, Loss: 1.6652449369430542
Epoch 2, Batch 200, Loss: 1.6966990232467651
Epoch 2, Batch 300, Loss: 1.6494026184082031
Epoch 2, Batch 400, Loss: 1.6629588603973389
Epoch 2, Batch 500, Loss: 1.6540671586990356
Epoch 2, Batch 600, Loss: 1.6844050884246826
Epoch 2, Batch 700, Loss: 1.6573057174682617
Epoch 2, Batch 800, Loss: 1.6480507850646973
Epoch 2, Batch 900, Loss: 1.627964735031128
Epoch 3, Batch 0, Loss: 1.68733811378479
Epoch 3, Batch 100, Loss: 1.637851595878601
Epoch 3, Batch 200, Loss

In [6]:
# Проверка обученной модели
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        _, predicted = torch.max(output, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

accuracy = correct / total
print(f'Accuracy on the training set: {100 * accuracy:.2f}%')

Accuracy on the training set: 89.01%
